In [1]:
using CompTime, InteractiveUtils

In [2]:
include("MacroUtils.jl")
using .MacroUtils: cleanup

In [3]:
function pw_rec(n, x)
    if iszero(n)
        one(x)
    else
        x * pw_rec(n - 1, x)
    end
end

pw_rec (generic function with 1 method)

In [4]:
Tuple(pw_rec(i, "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [5]:
function pw_loop(n, x)
    r = one(x)
    for i in 1:n
        r = r * x
    end
    return r
end

pw_loop (generic function with 1 method)

In [6]:
Tuple(pw_loop(i, "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [7]:
@ct_enable function pw_ct(@ct(n), x::T) where {T}
    r = @ct(one(T))
    @ct_ctrl for i in 1:n
        r = r * x
    end
    return r
end

runtime (generic function with 1 method)

In [8]:
pw_ct(n, x) = pw_ct(Val{n}, x)

pw_ct (generic function with 2 methods)

In [9]:
Tuple(pw_ct(i, "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [10]:
Tuple(runtime(pw_ct, i, "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [11]:
Tuple(comptime(pw_ct, Val{i}, "Abc") for i in 0:3)

("", "Abc", "AbcAbc", "AbcAbcAbc")

In [12]:
@code_typed pw_ct(Val{3}, 10)

CodeInfo(
1 ─ %1 = Base.mul_int(1, x)::Int64
│   %2 = Base.mul_int(%1, x)::Int64
│   %3 = Base.mul_int(%2, x)::Int64
└──      return %3
) => Int64

In [13]:
@code_lowered runtime(pw_ct, 3, 10)

CodeInfo(
1 ─       r = Main.one($(Expr(:static_parameter, 1)))
│   %2  = 1:n
│         @_5 = Base.iterate(%2)
│   %4  = @_5 === nothing
│   %5  = Base.not_int(%4)
└──       goto #4 if not %5
2 ┄ %7  = @_5
│         i = Core.getfield(%7, 1)
│   %9  = Core.getfield(%7, 2)
│         r = r * x
│         @_5 = Base.iterate(%2, %9)
│   %12 = @_5 === nothing
│   %13 = Base.not_int(%12)
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return r
)

In [14]:
debug(pw_ct, Val{3}, 10) |> cleanup

quote
    r = 1
    r = r * x
    r = r * x
    r = r * x
    return r
end